In [ ]:
# | default_exp nets/maxvit_3d

# Imports

In [ ]:
# | export

import torch
from torch import nn

from vision_architectures.blocks.cnn import CNNBlock3D, CNNBlockConfig
from vision_architectures.blocks.mbconv_3d import MBConv3D, MBConv3DConfig
from vision_architectures.blocks.transformer import Attention3DWithMLPConfig
from vision_architectures.nets.swinv2_3d import SwinV23DLayer
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import CustomBaseModel, Field, model_validator
from vision_architectures.utils.rearrange import rearrange_channels

# Config

In [ ]:
# | export


class MaxViT3DStem0Config(CNNBlockConfig):
    in_channels: int
    kernel_size: int = 3
    dim: int
    depth: int = 2

    out_channels: None = Field(None, description="This is defined by dim")


class MaxViT3DBlockConfig(MBConv3DConfig, Attention3DWithMLPConfig):
    window_size: tuple[int, int, int]
    modify_dims: bool = False  # Used in the last block of stems
    out_dim_ratio: int = 2  # Used only if modify_dims is True


class MaxViT3DStemConfig(MaxViT3DBlockConfig):
    depth: int


class MaxViT3DEncoderConfig(CustomBaseModel):
    stem0: MaxViT3DStem0Config
    stems: list[MaxViT3DStemConfig]

    @model_validator(mode="after")
    def validate(self):
        super().validate()
        assert self.stem0.dim == self.stems[0].dim, "Stem0 dim should be equal to the first stem dim"
        for i in range(1, len(self.stems)):
            assert (
                self.stems[i - 1].dim * self.stems[i - 1].out_dim_ratio == self.stems[i].dim
            ), "Stem dims should match"
        return self

# Architecture

In [ ]:
# | export


class MaxViT3DStem0(nn.Module):
    def __init__(self, config: MaxViT3DStem0Config = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = MaxViT3DStem0Config.model_validate(config | kwargs)

        self.layers = nn.ModuleList()
        self.layers.append(
            CNNBlock3D(
                self.config.model_dump() | dict(out_channels=self.config.dim),
                checkpointing_level,
            )
        )
        for i in range(self.config.depth - 1):
            self.layers.append(
                CNNBlock3D(
                    self.config.model_dump()
                    | dict(
                        in_channels=self.config.dim,
                        out_channels=self.config.dim,
                        normalization=self.config.normalization if i < self.config.depth - 1 else None,
                        activation=self.config.activation if i < self.config.depth - 1 else None,
                    ),
                    checkpointing_level,
                )
            )

        self.checkpointing_level2 = ActivationCheckpointing(2, checkpointing_level)

    def _forward(self, x: torch.Tensor, channels_first: bool = True):
        # x: (b, [in_channels], z, y, x, [in_channels])

        x = rearrange_channels(x, channels_first, True)
        # Now x is (b, in_channels, z, y, x)

        for layer in self.layers:
            x = layer(x)
            # (b, dim, z1, y1, x1)

        x = rearrange_channels(x, True, channels_first)
        # (b, [dim], z1, y1, x1, [dim])

        return x

    def forward(self, *args, **kwargs):
        return self.checkpointing_level2(self._forward, *args, **kwargs)

In [ ]:
test = MaxViT3DStem0(in_channels=4, dim=8)
display(test)

sample_input = torch.randn(1, 4, 32, 32, 32)
test(sample_input).shape

In [ ]:
# | export


class MaxViT3DBlockAttention(SwinV23DLayer):
    pass

In [ ]:
test = MaxViT3DBlockAttention(
    dim=12,
    num_heads=2,
    window_size=(4, 4, 4),
)
display(test)

sample_input = torch.randn(1, 12, 20, 20, 20)
test(sample_input).shape

In [ ]:
# | export


class MaxViT3DGridAttention(SwinV23DLayer):
    """Perform grid attention on the input tensor.

    Note that the grid attention implementation differs from the paper where the image is being partitioned based on the
    window size and not based on the number of windows. For example:

    Let us say the input is

    .. code-block:: text

        A1 A2 A3 A4 A5 A6
        B1 B2 B3 B4 B5 B6
        C1 C2 C3 C4 C5 C6
        D1 D2 D3 D4 D5 D6

    Let us say the window size is 2x2. The grid attention will be performed on the following 6 windows:

    .. code-block:: text

        A1 A4  A2 A5  A3 A6
        C1 C4  C2 C5  C3 C6

        B1 B4  B2 B5  B3 B6
        D1 D4  D2 D5  D3 D6

    According to the paper, my understanding is that attention should have been applied on the following 4 windows:

    .. code-block:: text

        A1 A3 A5  A2 A4 A6
        B1 B3 B5  B2 B4 B6

        C1 C3 C5  C2 C4 C6
        D1 D3 D5  D2 D4 D6

    i.e. the first token of all 2x2 windows in block attention, the second token of all 2x2 windows in block attention
    and so on.

    This has been implemented different so as to limit the number of tokens to be attended to in a window, as if
    utilized as per the paper, since 3D inputs are usually very large, the number of total windows in block attention
    would be very large, leading to a very large number of tokens to attend to in each window in grid attention.

    It would also cause problems when estimating the position embeddings as the grid size of the position embeddings
    would vary very with every input size.
    """

    @staticmethod
    def _get_rearrange_patterns():
        forward_pattern = (
            "b (window_size_z num_windows_z) (window_size_y num_windows_y) (window_size_x num_windows_x) dim -> "
            "(b num_windows_z num_windows_y num_windows_x) window_size_z window_size_y window_size_x dim "
        )
        reverse_pattern = (
            "(b num_windows_z num_windows_y num_windows_x) window_size_z window_size_y window_size_x dim -> "
            "b (window_size_z num_windows_z) (window_size_y num_windows_y) (window_size_x num_windows_x) dim"
        )
        return forward_pattern, reverse_pattern

In [ ]:
test = MaxViT3DGridAttention(
    dim=12,
    num_heads=2,
    window_size=(4, 4, 4),
)
display(test)

sample_input = torch.randn(1, 12, 20, 20, 20)
test(sample_input).shape

In [ ]:
# | export


class MaxViT3DBlock(nn.Module):
    def __init__(self, config: MaxViT3DBlockConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = MaxViT3DBlockConfig.model_validate(config | kwargs)

        mbconv_kwargs = {}
        out_dim = self.config.dim
        if self.config.modify_dims:
            out_dim = self.config.dim * self.config.out_dim_ratio
            mbconv_kwargs["stride"] = 2
            mbconv_kwargs["padding"] = 1
            mbconv_kwargs["out_dim"] = out_dim

        self.mbconv = MBConv3D(self.config.model_dump(), checkpointing_level=checkpointing_level, **mbconv_kwargs)

        # modify dim in the config
        self.config = MaxViT3DBlockConfig.model_validate(self.config | {"dim": out_dim})

        self.block_attention = MaxViT3DBlockAttention(self.config.model_dump(), checkpointing_level=checkpointing_level)
        self.grid_attention = MaxViT3DGridAttention(self.config.model_dump(), checkpointing_level=checkpointing_level)

        self.checkpointing_level3 = ActivationCheckpointing(3, checkpointing_level)

    def _forward(self, x: torch.Tensor, channels_first: bool = True):
        # x: (b, [dim], z, y, x, [dim])

        x = self.mbconv(x, channels_first)  # this runs in channels_first format internally
        # (b, [dim], z1, y1, x1, [dim])

        x = rearrange_channels(x, channels_first, False)
        # (b, z1, y1, x1, dim)

        x = self.block_attention(x, channels_first=False)
        # (b, z1, y1, x1, dim)
        x = self.grid_attention(x, channels_first=False)
        # (b, z1, y1, x1, dim)

        x = rearrange_channels(x, False, channels_first)
        # (b, [dim], z1, y1, x1, [dim])

        return x

    def forward(self, *args, **kwargs):
        return self.checkpointing_level3(self._forward, *args, **kwargs)

In [ ]:
test = MaxViT3DBlock(dim=12, num_heads=3, window_size=(4, 4, 4))
display(test)

sample_input = torch.randn(2, 12, 20, 20, 20)
test(sample_input).shape

In [ ]:
test = MaxViT3DBlock(dim=12, num_heads=3, window_size=(2, 2, 2), modify_dims=True)
display(test)

sample_input = torch.randn(2, 12, 20, 20, 20)
test(sample_input).shape

In [ ]:
# | export


class MaxViT3DStem(nn.Module):
    def __init__(
        self, config: MaxViT3DStemConfig = {}, checkpointing_level: int = 0, dont_downsample: bool = False, **kwargs
    ):
        super().__init__()

        self.config = MaxViT3DStemConfig.model_validate(config | kwargs)

        self.blocks = nn.ModuleList(
            MaxViT3DBlock(
                self.config.model_dump(),
                checkpointing_level=checkpointing_level,
                modify_dims=True if i == self.config.depth - 1 and not dont_downsample else False,
            )
            for i in range(self.config.depth)
        )

        self.checkpointing_level4 = ActivationCheckpointing(4, checkpointing_level)

    def _forward(self, x: torch.Tensor, channels_first: bool = True):
        # x: (b, [dim], z, y, x, [dim])
        for layer in self.blocks:
            x = layer(x, channels_first)
        return x

    def forward(self, *args, **kwargs):
        return self.checkpointing_level4(self._forward, *args, **kwargs)

In [ ]:
test = MaxViT3DStem(
    dim=12,
    num_heads=3,
    depth=3,
    window_size=(2, 2, 2),
)
display(test)

sample_input = torch.randn(2, 12, 20, 20, 20)
test(sample_input).shape

In [ ]:
test = MaxViT3DStem(
    dim=12,
    num_heads=3,
    depth=3,
    window_size=(2, 2, 2),
    dont_downsample=True,
)
display(test)

sample_input = torch.randn(2, 12, 20, 20, 20)
test(sample_input).shape

In [ ]:
# | export


class MaxViT3DEncoder(nn.Module):
    def __init__(self, config: MaxViT3DEncoderConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = MaxViT3DEncoderConfig.model_validate(config | kwargs)

        self.stems = nn.ModuleList([])
        self.stems.append(MaxViT3DStem0(self.config.stem0, checkpointing_level))
        for i, stem_config in enumerate(self.config.stems):
            self.stems.append(
                MaxViT3DStem(stem_config, checkpointing_level, dont_downsample=i == len(self.config.stems) - 1)
            )

        self.checkpointing_level5 = ActivationCheckpointing(5, checkpointing_level)

    def _forward(self, x: torch.Tensor, return_intermediates: bool = False, channels_first: bool = True):
        # x: (b, [in_channels], z, y, x, [in_channels])
        features = []
        for stem in self.stems:
            x = stem(x, channels_first)
            features.append(x)
        if return_intermediates:
            return x, features
        return x

    def forward(self, *args, **kwargs):
        return self.checkpointing_level5(self._forward, *args, **kwargs)

In [ ]:
test = MaxViT3DEncoder(
    stem0={
        "in_channels": 1,
        "dim": 12,
    },
    stems=[
        {
            "dim": 12,
            "num_heads": 3,
            "window_size": (2, 2, 2),
            "depth": 3,
        },
        {
            "dim": 24,
            "num_heads": 3,
            "window_size": (2, 2, 2),
            "depth": 3,
        },
    ],
)
display(test)

sample_input = torch.randn(2, 1, 32, 32, 32)
test(sample_input).shape

# nbdev

In [ ]:
!nbdev_export